In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import pickle
import os
from matplotlib import pyplot as plt


# Convert dict to Yolo format

In [ ]:
sign_name_carolo_trans = {
	0: 0,
	1: 1,
	2: 2,
	3: 3,
	4: 4,
	5: 5,
	9: 6,
	12: 7,
	13: 8,
	14: 9,
	33: 10,
	34: 11,
	38: 12,
	41: 13,
	43: 30,
	100: 14,
	101: 15,
	102: 16,
	103: 17,
	104: 18,
	105: 19,
	106: 20,
	107: 21,
	108: 22,
	109: 23,
	110: 24,
	111: 25,
	112: 26,
	113: 27,
	114: 28,
	115: 29
}

In [ ]:
sign_name_carolo_trans_pretrain = {
	0: 0,
	1: 0,
	2: 0,
	3: 0,
	4: 0,
	5: 0,
	9: 1,
	12: 2,
	13: 3,
	14: 4,
	33: 5,
	34: 6,
	38: 7,
	41: 8,
	43: 22,
	100: 9,
	101: 10,
	102: 11,
	103: 12,
	104: 13,
	105: 14,
	106: 15,
	107: 16,
	108: 17,
	109: 18,
	110: 19,
	111: 20,
	112: 0,
	113: 0,
	114: 0,
	115: 21
}

In [ ]:
sign_name_carolo_dict_new = {
    0: '20 Zone Anfang (speed limit 20 start)',
	1: '30 Zone Anfang (speed limit 30 start)',
	2: '50 Zone Anfang (speed limit 50 start)',
	3: '60 Zone Anfang (speed limit 60 start)',
	4: '70 Zone Anfang (speed limit 70 start)',
	5: '80 Zone Anfang (speed limit 80 start)',
	6: 'Absolutes Ueberholverbot Anfang (no passing zone start)',
	7: 'Vorfahrtstrasse (priority on next intersections)',
	8: 'Vorfahrt Gewaehren (give way to incoming)',
	9: 'Stop Zeichen (stop for at least 3 sec)',
	10: 'Vorgeschriebene Fahrtrichtung Rechts (turn right on intersection)',
	11: 'Vorgeschriebene Fahrtrichtung Links (turn left on intersection)',
	12: 'Vorgeschriebene Vorbeifahrt rechts (pass by on right)',
	13: 'Absolutes Ueberholverbot Ende (no passing zone end)',
	30: 'Nullklasse (zero class)',
	14: 'Gegenverkehr Vorrang gewaehren (Barred area, let oncoming pass)',
	15: 'Zone Ende (end of speed limit)',
	16: 'Fussgaengerueberweg (crosswalk)',
	17: 'Richtungstafel Kurve links (sharp turn left)',
	18: 'Richtungstafel Kurve links (even sharper turn left)',
	19: 'Richtungstafel Kurve rechts (sharp turn right)',
	20: 'Richtungstafel Kurve rechts (even sharper turn right)',
	21: 'Gefaelle 10% (downhill 10%)',
	22: 'Steigung 10% (uphill 10%)',
	23: 'Parken Bereich (parking zone)',
	24: 'Kraftfahrtstrasse Anfang (expressway begin)',
	25: 'Kraftfahrtstrasse Ende (expressway end)',
	26: '10 Zone Anfang (speed limit 10 start)',
	27: '40 Zone Anfang (speed limit 40 start)',
	28: '90 Zone Anfang (speed limit 90 start)',
	29: 'Pedestrian'
}

In [ ]:
def transform_coordinates(x1,y1,x2,y2,dimX,dimY):
    centerX=(x1+x2)/2
    centerY=(y1+y2)/2
    width=x2-x1
    height=y2-y1
    #to relative values:
    centerX=centerX/dimX
    centerY=centerY/dimY
    width=width/dimX
    height=height/dimY
    
    return centerX,centerY,width,height
    

# Creating labelfiles

In [ ]:
frames = []
datafiles=[]
files=[]
oldFile=None
num_Zero=2000
counter=0
zero_class=sign_name_carolo_trans.get(43)
basepath='/data/Images/'
#datafiles.append(basepath+'test_environment')
datafiles.append(basepath+'signs001_manually_labeled')
datafiles.append(basepath+'signs003_manually_labeled')
datafiles.append(basepath+'signs004_manually_labeled')
datafiles.append(basepath+'signs006_bosch_manually_labeled')
labelcounter=np.zeros((20,))
for folder in datafiles:
    for filename in os.listdir(folder):
        if filename.endswith('.csv'):
            print(filename)
            df = pd.read_csv(folder+'/'+filename)
            df['ClassId'].replace(sign_name_carolo_trans, inplace=True)
            for index, row in df.iterrows():           
                classid=row['ClassId']
                centerX,centerY,width,height=transform_coordinates(row['Roi.X1'],row['Roi.Y1'],row['Roi.X2'],row['Roi.Y2'],row['Width'],row['Height'])
                file=row['Filename'].split('.')[0]
                labelcounter[classid]+=1
                if classid!=zero_class:
                    if file==oldFile:
                        #print(centerX,centerY,width,height)
                        f = open(folder+'/'+file+'.txt',"a") 
                        f.write(str(classid)+' '+str(centerX)+' '+str(centerY)+' '+str(width)+' '+str(height)+'\n')
                        f.close()
                    else:
                        f = open(folder+'/'+file+'.txt',"w") 
                        f.write(str(classid)+' '+str(centerX)+' '+str(centerY)+' '+str(width)+' '+str(height)+'\n')
                        f.close()
                        #print(folder+'/'+file)
                        files.append(folder+'/'+file)
                elif classid==20 and counter<num_Zero:
                    f = open(folder+'/'+file+'.txt',"w") 
                    f.close()
                    counter+=1
                    files.append(folder+'/'+file)
                oldFile=file
                
                #frames_added_april.append(pd.read_csv(filename))

In [ ]:
plt.bar(np.labelcounter(len(s)),s)
plt.show()

# Train & Test files

In [ ]:
#from sklearn.model_selection import train_test_split

In [ ]:
#X_train,X_test=train_test_split(files, shuffle=False)

num_of_rows = round((len(files)) * 0.8)
files_np=np.array(files)
np.random.shuffle(files_np) 
X_train= files_np[:num_of_rows] 
X_test = files_np[num_of_rows:] 

file = open(basepath+'train.txt',"w")
for line in X_train:
    file.write(str(line)+'.png\n')
file.close()
    
file = open(basepath+'test.txt',"w")
for line in X_test:
    file.write(str(line)+'.png\n')
file.close()
    

In [ ]:
file = open(basepath+'signs.names',"w")
for num in range(len(sign_name_carolo_trans)-1):
    file.write(str(sign_name_carolo_dict_new.get(num))+'.png\n')
file.close()

In [ ]:
with open('../sign_recognition/dict/sign_names_dict.pkl', 'rb') as f:
    sign_names = pickle.load(f)

In [ ]:
sign_names

In [ ]:
len(sign_name_carolo_trans)